In [1]:
import torch
import pytorch_lightning as pl

In [2]:
import config
from dataset import DataModule
from model import ClassificationModel

In [3]:
torch.random.manual_seed(config.RANDOM_SEED)
pl.seed_everything(config.RANDOM_SEED)

Seed set to 42


42

In [4]:
dm = DataModule(
    data_path=config.DATA_PATH,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    train_test_ratio=config.TRAIN_TEST_RATIO,
    train_val_ratio=config.TRAIN_VAL_RATIO,
)

model = ClassificationModel(learning_rate=config.LEARNING_RATE)

trainer = pl.Trainer(
    accelerator=config.ACCELERATOR,
    devices=config.DEVICES,
    min_epochs=config.MIN_EPOCHS,
    max_epochs=config.MAX_EPOCHS,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/evlko/Documents/GitHub/Game-Interface-Flows-ML/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [5]:
trainer.fit(model, dm)
trainer.validate(model, dm)
trainer.test(model, dm)


  | Name     | Type           | Params
--------------------------------------------
0 | loss_fn  | BCELoss        | 0     
1 | accuracy | BinaryAccuracy | 0     
2 | conv1    | Conv2d         | 448   
3 | conv2    | Conv2d         | 4.6 K 
4 | pool     | MaxPool2d      | 0     
5 | fc1      | Linear         | 8.4 M 
6 | fc2      | Linear         | 65    
--------------------------------------------
8.4 M     Trainable params
0         Non-trainable params
8.4 M     Total params
33.575    Total estimated model params size (MB)


/Users/evlko/Documents/GitHub/Game-Interface-Flows-ML/.venv/lib/python3.12/site-packages/torch/functional.py:905: UserWarning: MPS: _unique2 op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performace implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Unique.mm:323.)
  output, inverse_indices, counts = torch._unique2(
/Users/evlko/Documents/GitHub/Game-Interface-Flows-ML/.venv/lib/python3.12/site-packages/torchmetrics/utilities/compute.py:52: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Indexing.mm:283.)
  denom[denom == 0.0] = 1
/Users/evlko/Documents/GitHub/Game-Interface-Flows-ML/.venv/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (1

Epoch 9: 100%|██████████| 1/1 [00:01<00:00,  0.67it/s, v_num=57]

`Trainer.fit` stopped: `max_epochs=10` reached.


Validation DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 29.11it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy                  1.0
        val_loss           0.014504649676382542
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 17.48it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy                0.75
        test_loss           0.4346815943

[{'test_loss': 0.43468159437179565, 'test_accuracy': 0.75}]

In [6]:
script = model.to_torchscript()
torch.jit.save(script, config.MODEL_PATH)